In [1]:
# STEP 1: Upload data
# - small_data.zip - About 17 mins

In [27]:
!conda install fastbook

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - fastbook

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [1]:
!pip install -Uqq fastbook
import fastbook
from fastai.vision.all import *
from fastai.text.all import *
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Dict, Mapping, Iterable, Optional
import torchvision.transforms as T
import pickle 

/opt/conda/lib/python3.8/site-packages/fastbook/__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
## DATA ##

In [3]:
import zipfile
def extract_data(zipfilename):
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
        zip_ref.extractall(".")
# extract_data("small_data.zip")
# extract_data("data_full.zip")

In [4]:
root_dir = Path('.')

In [5]:
def get_image_files_train_valid(*args, **kwargs):
  return get_image_files(root_dir/'data_full'/'train') + get_image_files(root_dir/'data_full'/'valid')

In [6]:
metadata = pd.read_csv(root_dir/'data.csv', index_col=0)

In [7]:
def get_y(path):
  p = str(path).rstrip(".jpg")
  if p.endswith(" (1)"): p = p[:-4]
  idx = int(re.search(r'(\d*)$', p).group(1))
  return metadata.country[idx]

In [8]:
def splitter(paths, small_set=False):
  train_idx = []
  valid_idx = []
  for i, p in enumerate(paths):
    # if small_set and get_y(p) not in small_set_labels:
    #   continue
    if p.parent.name == 'valid': valid_idx.append(i)
    else: train_idx.append(i)
  return train_idx, valid_idx

In [9]:
def get_dls(bs, size, min_scale):
  item_tfms = [] if size == 224 else [Resize(448)]
  normalize_tfm = Normalize.from_stats(*imagenet_stats)
  block = DataBlock(
      get_items=get_image_files_train_valid,
      get_y=get_y,
      blocks=(ImageBlock, CategoryBlock),
      splitter=splitter,
      item_tfms=item_tfms,
      batch_tfms=[
        *aug_transforms(min_scale=min_scale),
        normalize_tfm,
      ]
  )
  im_path = Path()
  return block.dataloaders(im_path, bs=bs)

In [10]:
## MODEL ##

In [11]:
def get_model(dls, arch, dropout_amt):
  arch_to_feats = {resnet34: 512, resnet50: 2048, resnet101: 2048, resnet152: 2048}
  return nn.Sequential(
      create_body(arch, cut=-2),
      create_head(arch_to_feats[arch], len(dls.vocab), lin_ftrs=[512], ps=dropout_amt),
  )

In [12]:
## TRAIN ## 

In [13]:
dls = get_dls(bs=32, size=448, min_scale=1.0)

In [14]:
len(dls.vocab), dls.vocab

(87,
 ['ae', 'al', 'ar', 'as', 'at', 'au', 'bd', 'be', 'bg', 'bo', 'br', 'bt', 'bw', 'ca', 'ch', 'cl', 'cn', 'co', 'cz', 'de', 'dk', 'do', 'ec', 'ee', 'eg', 'es', 'fi', 'fo', 'fr', 'gb', 'gh', 'gl', 'gr', 'gt', 'hk', 'hr', 'hu', 'id', 'ie', 'il', 'in', 'is', 'it', 'jo', 'jp', 'ke', 'kg', 'kh', 'kr', 'lk', 'ls', 'lt', 'lv', 'mg', 'mk', 'mn', 'mx', 'my', 'ng', 'nl', 'no', 'nz', 'pe', 'ph', 'pk', 'pl', 'pr', 'pt', 'ro', 'rs', 'ru', 'se', 'sg', 'si', 'sk', 'sn', 'sz', 'th', 'tn', 'tr', 'tw', 'ua', 'ug', 'us', 'uy', 'vn', 'za'])

In [15]:
model = get_model(dls, resnet101, dropout_amt=0.)

In [16]:
learn = Learner(dls, model, CrossEntropyLossFlat(), metrics=accuracy)
learn.freeze()
learn.lr_find()

RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 31.75 GiB total capacity; 9.79 GiB already allocated; 23.50 MiB free; 9.87 GiB reserved in total by PyTorch)

In [23]:
learn.fit_one_cycle(10, 4e-4)

epoch,train_loss,valid_loss,accuracy,time
0,2.405946,2.303111,0.430102,11:25
1,2.191706,2.271789,0.404141,11:43
2,1.900909,2.032244,0.464098,11:34
3,1.656531,1.647158,0.540641,11:16
4,1.519900,1.523049,0.566189,11:11
5,1.247465,1.363458,0.607809,11:14
6,1.110256,1.365860,0.613475,11:44
7,0.891497,1.248786,0.644792,11:37
8,0.768882,1.225055,0.656846,11:14
9,0.679264,1.221152,0.653446,11:24


In [24]:
torch.save(learn.model.state_dict, root_dir/"state_dict_country_resnet101_448_frozen_5_30")